In [22]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.decomposition import PCA
import csv

In [23]:
#Read Excel file in pandas
df = pd.read_excel('C:\\Users\\fleet\\Downloads\\Telegram Desktop\\five_groups_4.xlsx')
print(df.isnull().sum())

Группа                             0
3-OH Anthranillic acid Results    55
5-hydroxytryptophan Results       47
5-Methoxytryptamine Results       43
adenosin Results                  36
                                  ..
C5-OH Results                     43
C6 Results                        34
C6-DC Results                     43
C8 Results                        41
C8-1 Results                      44
Length: 93, dtype: int64


In [24]:
df['Группа'].replace(['ИБС','Контроль'],[0,1],inplace=True)
print(df.isnull().sum())

Группа                             0
3-OH Anthranillic acid Results    55
5-hydroxytryptophan Results       47
5-Methoxytryptamine Results       43
adenosin Results                  36
                                  ..
C5-OH Results                     43
C6 Results                        34
C6-DC Results                     43
C8 Results                        41
C8-1 Results                      44
Length: 93, dtype: int64


In [25]:
x = df.iloc[:,1:]
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='median')
non = imputer.fit_transform(x)
x = pd.DataFrame(non, columns=x.columns)

y = df.iloc[:,0]
df = pd.concat([y,x],axis=1)
print(df)

     Группа  3-OH Anthranillic acid Results  5-hydroxytryptophan Results  \
0         0                        0.162319                     0.201433   
1         0                        0.011952                    -1.886624   
2         0                        0.024094                     2.488508   
3         0                       -0.031603                     0.201433   
4         0                        0.052937                    -1.901449   
..      ...                             ...                          ...   
155       1                       -0.012791                    -1.401126   
156       1                        0.011952                    -1.358742   
157       1                       -0.020162                     0.232693   
158       1                        0.237833                     0.371351   
159       1                        0.010848                    -1.374326   

     5-Methoxytryptamine Results  adenosin Results  ADMA Results  \
0                  

In [26]:
print('ИБС', sum(df['Группа'] == 0),\
        '\nКонтроль', sum(df['Группа'] == 1))

ИБС 82 
Контроль 78


In [27]:
#Data visualization
y.value_counts()

0    82
1    78
Name: Группа, dtype: int64

In [28]:
#Scaling the data
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_norm = sc.fit_transform(x)
x_norm = pd.DataFrame(x_norm, columns=x.columns)
x_norm.describe()

,3-OH Anthranillic acid Results,5-hydroxytryptophan Results,5-Methoxytryptamine Results,adenosin Results,ADMA Results,Alanine Results,Antranillic acid Results,Arginine Results,Asparagine Results,Aspartic acid Results,...,C3 Results,C4 Results,C5 Results,C5-1 Results,C5-DC Results,C5-OH Results,C6 Results,C6-DC Results,C8 Results,C8-1 Results
count,1.600000e+02,1.600000e+02,1.600000e+02,1.600000e+02,1.600000e+02,1.600000e+02,1.600000e+02,1.600000e+02,1.600000e+02,1.600000e+02,...,1.600000e+02,1.600000e+02,1.600000e+02,1.600000e+02,1.600000e+02,1.600000e+02,1.600000e+02,160.000000,1.600000e+02,160.000000
mean,5.551115e-17,-2.220446e-17,-1.443290e-16,1.554312e-16,-6.661338e-17,-7.771561e-17,8.881784e-17,-2.498002e-17,2.775558e-17,-2.220446e-17,...,2.636780e-17,-2.220446e-17,3.330669e-17,-1.776357e-16,4.996004e-17,7.771561e-17,2.220446e-17,0.000000,3.330669e-17,0.000000
std,1.003140e+00,1.003140e+00,1.003140e+00,1.003140e+00,1.003140e+00,1.003140e+00,1.003140e+00,1.003140e+00,1.003140e+00,1.003140e+00,...,1.003140e+00,1.003140e+00,1.003140e+00,1.003140e+00,1.003140e+00,1.003140e+00,1.003140e+00,1.003140,1.003140e+00,1.003140
min,-1.093081e+00,-1.676048e+00,-2.288304e+00,-2.486826e+00,-1.873831e+00,-1.892330e+00,-1.497337e+00,-1.977250e+00,-1.992605e+00,-1.891899e+00,...,-1.599689e+00,-1.766151e+00,-1.829161e+00,-2.092541e+00,-1.616834e+00,-2.134435e+00,-1.749026e+00,-1.842202,-2.047365e+00,-1.190910
25%,-6.604736e-01,-7.092303e-01,-5.305670e-01,-5.898326e-01,-4.985636e-01,-6.516251e-01,-5.400922e-01,-7.237149e-01,-5.148026e-01,-4.137517e-01,...,-6.764971e-01,-5.483171e-01,-6.527290e-01,-5.893218e-01,-5.897889e-01,-6.000092e-01,-6.270626e-01,-0.658685,-5.755564e-01,-0.502819
50%,-1.790911e-01,-2.735603e-02,3.066034e-02,3.886992e-02,-1.298232e-01,-9.978502e-02,-1.213089e-01,-4.595130e-02,-1.001918e-01,-1.291964e-01,...,-1.494542e-01,-1.294412e-01,-8.085120e-02,3.686387e-02,-2.511804e-01,-1.892066e-02,-1.618503e-01,-0.072919,-2.908521e-02,-0.273817
75%,6.623938e-02,2.439292e-01,6.105538e-01,7.945650e-01,5.266204e-01,6.195965e-01,1.493364e-01,6.035207e-01,3.361636e-01,4.055662e-01,...,4.296982e-01,2.599881e-01,5.247482e-01,4.659832e-01,4.739290e-01,5.954383e-01,4.610971e-01,0.251221,5.774654e-01,0.257243
max,4.216687e+00,2.647618e+00,2.022664e+00,1.886474e+00,2.494531e+00,2.712852e+00,2.981089e+00,2.314502e+00,2.897569e+00,2.572903e+00,...,2.928345e+00,2.935113e+00,2.460606e+00,2.300735e+00,2.885695e+00,2.542347e+00,2.625300e+00,2.610742,2.330761e+00,4.079332


In [29]:
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report,roc_curve,roc_auc_score,precision_score,recall_score,f1_score,precision_recall_curve
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVC

In [30]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x_norm,y,test_size=0.2,random_state=0, stratify=y)

In [31]:
#Performing models
models ={
    'Decision Tree':DecisionTreeClassifier(random_state=0),
    'Random Forest':RandomForestClassifier(random_state=0),
    'KNN':KNeighborsClassifier(),
    'Gradient Boosting': GradientBoostingClassifier(random_state=0),
    'SVM': SVC(random_state=0)
}
for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(x_train,y_train)
    skfold=StratifiedKFold(n_splits=3)
    scores = cross_val_score(model, x_norm, y, cv=skfold)

    y_train_pred = model.predict(x_train)
    y_test_pred = model.predict(x_test)
        #Test performance
    model_test_accuracy = accuracy_score(y_test,y_test_pred)
    model_test_precision = precision_score(y_test,y_test_pred, average='weighted', zero_division= 0)
    model_test_recall = recall_score(y_test,y_test_pred, average='weighted')
    model_test_f1 = f1_score(y_test,y_test_pred, average='weighted')

        #Train performance
    model_train_accuracy = accuracy_score(y_train,y_train_pred)
    model_train_precision = precision_score(y_train,y_train_pred, average='weighted', zero_division= 0)
    model_train_recall = recall_score(y_train,y_train_pred, average='weighted')
    model_train_f1 = f1_score(y_train,y_train_pred, average='weighted')
   
    print('='*35)
    print(list(models.keys())[i])

    print("CV mean score: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))
    print('='*35)

    print('Model performance for Training set:')
    print('Accuracy: {:.4f}'.format(model_train_accuracy))
    print('F1 Score: {:.4f}'.format(model_train_f1))
    print('Precision: {:.4f}'.format(model_train_precision))
    print('Recall: {:.4f}'.format(model_train_recall))

    print('-'*35)

    print('Model performance for Test set:')
    print('Accuracy: {:.4f}'.format(model_test_accuracy))
    print('F1 Score: {:.4f}'.format(model_test_f1))
    print('Precision: {:.4f}'.format(model_test_precision))
    print('Recall: {:.4f}'.format(model_test_recall))

    print('='*35)
    print('\n')


Decision Tree
CV mean score: 0.77 (+/- 0.22)
Model performance for Training set:
Accuracy: 1.0000
F1 Score: 1.0000
Precision: 1.0000
Recall: 1.0000
-----------------------------------
Model performance for Test set:
Accuracy: 0.8750
F1 Score: 0.8750
Precision: 0.8750
Recall: 0.8750




Random Forest
CV mean score: 0.94 (+/- 0.05)
Model performance for Training set:
Accuracy: 1.0000
F1 Score: 1.0000
Precision: 1.0000
Recall: 1.0000
-----------------------------------
Model performance for Test set:
Accuracy: 0.9375
F1 Score: 0.9373
Precision: 0.9444
Recall: 0.9375


KNN
CV mean score: 0.78 (+/- 0.21)
Model performance for Training set:
Accuracy: 0.9531
F1 Score: 0.9531
Precision: 0.9535
Recall: 0.9531
-----------------------------------
Model performance for Test set:
Accuracy: 0.9062
F1 Score: 0.9054
Precision: 0.9211
Recall: 0.9062


Gradient Boosting
CV mean score: 0.78 (+/- 0.33)
Model performance for Training set:
Accuracy: 1.0000
F1 Score: 1.0000
Precision: 1.0000
Recall: 1.0000
-----------------------------------
Model performance for Test set:
Accuracy: 0.9375
F1 Score: 0.9375
Precision: 0.9375
Recall: 0.9375


SVM
CV mean score: 0.91 (+/- 0.19)
Model performance for Training set:
Accuracy: 0.9922
F1 Score: 0.9922
Precision: 0.9923
Recall: 0.9922
-------------

In [32]:
#Hyperparamenter Training
rf_params = {'max_depth': [5,8,15, None, 10],
              'min_samples_split': [2, 5, 10],
              'n_estimators': [50, 100, 200, 500],
              'criterion':  ['gini', 'entropy'],
              'min_samples_leaf': [1, 2, 4],
              'class_weight': ['balanced', None]}
dt_params = { 'max_depth': [5,8,15, None, 10],
              'min_samples_split': [2, 5, 10],
              'max_leaf_nodes': [5, 10, 20],
              'min_samples_leaf': [1, 2, 4],
              'criterion':  ['gini', 'entropy']}
KNN_params = {'n_neighbors': [3,5,10],
              'weights': ['uniform', 'distance'],
              'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
              'p': [1,2],
              'metric': ['euclidean', 'manhattan'],
              'leaf_size': [20, 50, 100]}
GB_params = {'max_depth': [3,5,8,15, None],
              'min_samples_split': [2, 5, 10],
              'n_estimators': [50, 100, 200, 500],
              'loss': ['log_loss'],
              'criterion':  ['squared_error', 'friedman_mse'],
              'max_features': ['auto', 'sqrt', 'log2'],
              'min_samples_leaf': [1, 2, 4],
              'min_samples_split': [2, 5, 10],
              'max_leaf_nodes': [5, 10, 20]}
svc_params = {'C': [1, 10, 100, 1000],
              'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
              'gamma': ['scale', 'auto'],
              'decision_function_shape': ['ovo', 'ovr'],
              'class_weight': ['balanced', None]}
grid = {'learning_rate': [0.03, 0.1],
        'depth': [4, 6, 10],
        'l2_leaf_reg': [1, 3, 5, 7, 9]}


In [33]:
gridcv_models = [
                    ('RF', RandomForestClassifier(), rf_params),
                    ('SVC', SVC(), svc_params),
                    ]

In [34]:
""" #GridSearchCV hyperparameters pulling methods 
model_param = {}
for name,model,params in gridcv_models:
    random = GridSearchCV(estimator = model,\
                                param_grid = params,\
                               cv =3,\
                                verbose=2,\
                              n_jobs=-1)
    random.fit(x_train,y_train)
    model_param[name] = random.best_params_
    
for model_name in model_param:
    print(f"---------Best Params for {model_name}---------")
    print(model_param[model_name]) """

' #GridSearchCV hyperparameters pulling methods \nmodel_param = {}\nfor name,model,params in gridcv_models:\n    random = GridSearchCV(estimator = model,                                param_grid = params,                               cv =3,                                verbose=2,                              n_jobs=-1)\n    random.fit(x_train,y_train)\n    model_param[name] = random.best_params_\n    \nfor model_name in model_param:\n    print(f"---------Best Params for {model_name}---------")\n    print(model_param[model_name]) '

In [35]:
#Performing models
models ={
    'Random Forest':RandomForestClassifier(random_state = 0, n_estimators = 50, min_samples_split= 5,max_depth = 15, criterion = 'entropy',class_weight= 'balanced'),
    'Gradient Boosting':GradientBoostingClassifier(random_state=0, max_depth = 15, min_samples_split= 2, n_estimators = 200, loss = 'log_loss', criterion = 'squared_error', max_features = 'log2', min_samples_leaf= 2),
    'SVM': SVC(random_state = 0, C = 1, gamma = 'scale', kernel = 'rbf', decision_function_shape = 'ovo'),
}

for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(x_train,y_train)
    skfold=StratifiedKFold(n_splits=10, shuffle=True, random_state=0)
    scores = cross_val_score(model, x_norm, y, cv=skfold)

    y_train_pred = model.predict(x_train)
    y_test_pred = model.predict(x_test)
        #Test performance
    model_test_accuracy = accuracy_score(y_test,y_test_pred)
    model_test_precision = precision_score(y_test,y_test_pred, average='weighted', zero_division= 0)
    model_test_recall = recall_score(y_test,y_test_pred, average='weighted')
    model_test_f1 = f1_score(y_test,y_test_pred, average='weighted')

        #Train performance
    model_train_accuracy = accuracy_score(y_train,y_train_pred)
    model_train_precision = precision_score(y_train,y_train_pred, average='weighted', zero_division= 0)
    model_train_recall = recall_score(y_train,y_train_pred, average='weighted')
    model_train_f1 = f1_score(y_train,y_train_pred, average='weighted')
    
    print('='*35)
    print(list(models.keys())[i])

    print("CV mean score: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

    print('-'*35)

    print('Model performance for Training set:')
    print('Accuracy: {:.4f}'.format(model_train_accuracy))
    print('F1 Score: {:.4f}'.format(model_train_f1))
    print('Precision: {:.4f}'.format(model_train_precision))
    print('Recall: {:.4f}'.format(model_train_recall))

    print('-'*35)

    print('Model performance for Test set:')
    print('Accuracy: {:.4f}'.format(model_test_accuracy))
    print('F1 Score: {:.4f}'.format(model_test_f1))
    print('Precision: {:.4f}'.format(model_test_precision))
    print('Recall: {:.4f}'.format(model_test_recall))

    print('='*35)
    print('\n')

Random Forest
CV mean score: 0.94 (+/- 0.12)
-----------------------------------
Model performance for Training set:
Accuracy: 1.0000
F1 Score: 1.0000
Precision: 1.0000
Recall: 1.0000
-----------------------------------
Model performance for Test set:
Accuracy: 1.0000
F1 Score: 1.0000
Precision: 1.0000
Recall: 1.0000


Gradient Boosting
CV mean score: 0.97 (+/- 0.06)
-----------------------------------
Model performance for Training set:
Accuracy: 1.0000
F1 Score: 1.0000
Precision: 1.0000
Recall: 1.0000
-----------------------------------
Model performance for Test set:
Accuracy: 0.9688
F1 Score: 0.9687
Precision: 0.9706
Recall: 0.9688


SVM
CV mean score: 0.96 (+/- 0.10)
-----------------------------------
Model performance for Training set:
Accuracy: 0.9922
F1 Score: 0.9922
Precision: 0.9923
Recall: 0.9922
-----------------------------------
Model performance for Test set:
Accuracy: 1.0000
F1 Score: 1.0000
Precision: 1.0000
Recall: 1.0000




In [36]:
GB = GradientBoostingClassifier(random_state=0, max_depth = 15, min_samples_split= 2,\
                                n_estimators = 200, loss = 'log_loss', criterion = 'squared_error',\
                                max_features = 'log2', min_samples_leaf= 2)
GB.fit(x_train,y_train)
proba = GB.predict_proba(x_test)
predicted = GB.predict(x_test)
proba_r = np.round(proba, decimals=3)
y_test_groups = list(y_test)
for i in range(len(y_test_groups)):
    print('Probability of class {}'.format(proba_r[i]*100))
    print('Predicted class: {}'.format(predicted[i]))
    print('Real class: {}'.format(y_test_groups[i]))
    print('\n')

Probability of class [93.5  6.5]
Predicted class: 0
Real class: 0


Probability of class [99.8  0.2]
Predicted class: 0
Real class: 0


Probability of class [100.   0.]
Predicted class: 0
Real class: 0


Probability of class [ 1.8 98.2]
Predicted class: 1
Real class: 1


Probability of class [100.   0.]
Predicted class: 0
Real class: 0


Probability of class [100.   0.]
Predicted class: 0
Real class: 0


Probability of class [99.9  0.1]
Predicted class: 0
Real class: 0


Probability of class [ 0.1 99.9]
Predicted class: 1
Real class: 1


Probability of class [  0. 100.]
Predicted class: 1
Real class: 1


Probability of class [100.   0.]
Predicted class: 0
Real class: 0


Probability of class [100.   0.]
Predicted class: 0
Real class: 0


Probability of class [100.   0.]
Predicted class: 0
Real class: 0


Probability of class [ 0.1 99.9]
Predicted class: 1
Real class: 1


Probability of class [79.7 20.3]
Predicted class: 0
Real class: 1


Probability of class [  0. 100.]
Predicted class

In [37]:
#Classificatio report
print(classification_report(y_test, predicted))

              precision    recall  f1-score   support

           0       0.94      1.00      0.97        16
           1       1.00      0.94      0.97        16

    accuracy                           0.97        32
   macro avg       0.97      0.97      0.97        32
weighted avg       0.97      0.97      0.97        32



In [38]:
#AUC ROC
roc = roc_auc_score(y_test, predicted)
print(roc)

0.96875


In [39]:
from pycm import ConfusionMatrix
y_test = np.array(y_test)
predicted = np.array(predicted)
cm = ConfusionMatrix(actual_vector=y_test,predict_vector=predicted)
print(cm)

Predict  0        1        
Actual
0        16       0        

1        1        15       





Overall Statistics : 

95% CI                                                            (0.90846,1.02904)
ACC Macro                                                         0.96875
ARI                                                               0.87489
AUNP                                                              0.96875
AUNU                                                              0.96875
Bangdiwala B                                                      0.93945
Bennett S                                                         0.9375
CBA                                                               0.93934
CSI                                                               0.93934
Chi-Squared                                                       28.23529
Chi-Squared DF                                                    1
Conditional Entropy                                           

In [51]:
import shap
shap.initjs()

NameError: name '_C' is not defined

In [49]:
explainer = shap.Explainer(GB)
shap_values = explainer(x_test)
shap.plots.bar(shap_values)

NameError: name 'shap' is not defined